In [ ]:
#Requirement

In [ ]:
import pandas as pd
import numpy as np
import os
import gc
import random
import dash
import dash_core_components as dcc
import dash_html_components as html
import datetime
import warnings
import requests
import json
from collections import Counter
warnings.filterwarnings('ignore')

In [ ]:
# reading the collected manual keywords
keywords = pd.read_excel('keywordsscreens.xlsx')
sectors = pd.read_excel('sectorscreens.xlsx')
themes = pd.read_excel('theme_screeners.xlsx')
company = pd.read_excel('company_screener.xlsx')

In [ ]:
# Keep required columns only 
sectors1 = sectors[['Sector', 'keyword']]
sectors2 = sectors[['sector', 'keyword']]
themes = themes[['Theme', 'Alias']]
company = company[['Company', 'Aliases' ]]

keywords = keywords.values
sectors1 = sectors1.values
sectors2 = sectors2.values
themes = themes.values
company = company.values

In [ ]:
from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()
sector1_processor = KeywordProcessor()
sector2_processor = KeywordProcessor()
theme_processor = KeywordProcessor()
company_processor = KeywordProcessor()

for item in keywords:
    keyword_processor.add_keyword(str(item[1]), str(item[0]))   #(keyword, keyword generic name)
def keywordsf(x):
    kf = keyword_processor.extract_keywords(str(x))
    kf = list(kf)
    return kf

for item in sectors1:
    sector1_processor.add_keyword(str(item[1]), str(item[0]))   #(keyword, keyword generic name)
def sector1f(x):
    kf = sector1_processor.extract_keywords(str(x))
    kf = list(kf)
    return kf

for item in sectors2:
    sector2_processor.add_keyword(str(item[1]), str(item[0]))   #(keyword, keyword generic name)
def sector2f(x):
    kf = sector2_processor.extract_keywords(str(x))
    kf = list(kf)
    return kf

for item in themes:
    theme_processor.add_keyword(str(item[1]), str(item[0]))   #(keyword, keyword generic name)
def themef(x):
    kf = theme_processor.extract_keywords(str(x))
    kf = list(kf)
    return kf

for item in company:
    company_processor.add_keyword(str(item[1]), str(item[0]))   #(keyword, keyword generic name)
def companyf(x):
    kf = company_processor.extract_keywords(str(x))
    kf = list(kf)
    return kf


In [ ]:
#empty some memory
del keywords, sectors1, sectors2, themes, company
gc.collect()

In [ ]:
import twython
import json
import re
import time
from unidecode import unidecode

def setupTwythonAccount():

    twitter_access_credentials = []
    
    app_key = 'XXXX'
    app_secret = 'XXXX'
    oauth_token = "XXXX"
    oauth_token_secret="XXXX"
    twitter = twython.Twython(app_key, app_secret, oauth_token, oauth_token_secret)
    twitter_access_credentials.append(twitter)
    
    
    app_key = 'XXXX'
    app_secret = 'XXXX'
    oauth_token = "XXXX"
    oauth_token_secret="XXXX"
    twitter1 = twython.Twython(app_key, app_secret, oauth_token, oauth_token_secret)
    twitter_access_credentials.append(twitter1)
    
    app_key = 'XXXX'
    app_secret = 'XXXX'
    oauth_token = "XXXX"
    oauth_token_secret="XXXX"
    twitter2 = twython.Twython(app_key, app_secret, oauth_token, oauth_token_secret)
    twitter_access_credentials.append(twitter2)
    
    app_key = 'XXXX'
    app_secret = 'XXXX'
    oauth_token = "XXXX"
    oauth_token_secret="XXXX"
    twitter3 = twython.Twython(app_key, app_secret, oauth_token, oauth_token_secret)
    twitter_access_credentials.append(twitter3)
    return twitter_access_credentials

twitter_access_credentials = setupTwythonAccount()


In [ ]:
#basic helper funcutions
def links_text(x):
    if "http" in str(x):
        z = 1
    else:
        z = 0
    return z


def get_url(x):
    url = "https://twitter.com/i/web/status/" + str(x)
    return url

def retweets(x):    
    try:
        rt = x['retweeted_status']
    except:
        rt = []
    if len(rt)>0:
        z = 1
    else:
        z = 0
    return z

In [ ]:
#get latest 400 status for user provided  
def status_fn(x):    
    twitter = random.choice(twitter_access_credentials)  # makes sure we don't use same api keys again and again
    status = twitter.get_user_timeline(screen_name=x,count=200,tweet_mode='extended')
    if len(status)<300:
        try:
            twitter = random.choice(twitter_access_credentials)
            status2 = twitter.get_user_timeline(screen_name=x,count=200,tweet_mode='extended', max_id= status[-1]['id'])
        except:
            status2 =[]
    else:
        status2 =[]

    status_full = status + status2[1:]
    return status_full


In [ ]:
# process tweets from statuses collected
def get_tweets(status_full):
    df = []
    total_tweets = len(status_full)
    for i in range(total_tweets):
        temp = {}
        temp['created_at'] = status_full[i]['created_at']
        temp['id'] = status_full[i]['id']
        temp['full_text'] = status_full[i]['full_text']
        temp['favorite_count'] = status_full[i]['favorite_count']
        temp['retweet_count'] = status_full[i]['retweet_count']
        temp['RT_status'] = retweets(status_full[i])
        if temp['RT_status'] == 1:
            temp['retweet_count'] = 0
        temp['link'] = links_text(status_full[i]['full_text'])
        temp['tweet_len'] = len(status_full[i]['full_text'])
        
        #temp['hashtags'] = hasht(status_full[i]['entities']['hashtags'])
        df.append(temp)
    return df


In [ ]:
#get user information
def user_Details(status_full):
    temp ={}
    
    user = status_full[0]['user']
    
    temp['followers'] = user['followers_count']
    temp['following'] = user['friends_count']
    temp['verified'] = user['verified']
    temp['location'] = user['location']
    temp['name'] = user['name']
    temp['description'] = user['description']
    dtime = str(user['created_at'])
    new_datetime = datetime.datetime.strptime(dtime,'%a %b %d %H:%M:%S +0000 %Y')
    temp['created_at'] = str(user['created_at'])
    temp['statuses_count'] = user['statuses_count']
    temp['pp'] = user['profile_image_url_https']
    temp['days_lived'] = (datetime.datetime.today() - new_datetime).days
    temp['publication_daily'] = temp['statuses_count'] / temp['days_lived']
    return temp



In [ ]:
stock_pattern = re.compile(r"(\$\D*?)(?:\s|$)")
hash_pattern = re.compile(r"(\#\D*?)(?:\s|#)")

#small hack for if twitter results empty status
def df_views(df, cols):
    if len(df)==0:
        dft = pd.DataFrame(columns = cols)
    else:
        dft = pd.DataFrame(df)
        dft.columns = cols
    return dft


In [ ]:
def final_call(name):    
    status_full = status_fn(name) # get status
    data = get_tweets(status_full) #organising tweets
    df = pd.DataFrame(data)
    #doing calulations
    df['Top_content'] = df['favorite_count'] + df['retweet_count']
    top_content_id = df.sort_values(by = 'Top_content', ascending = False).head(2)['id'].values
    top_content_tweets_df = df.sort_values(by = 'Top_content', ascending = False).head(5)[["created_at", 'full_text', "favorite_count", 'retweet_count', 'id']]
    top_content_tweets_df.columns = ["Date", 'Tweet', "Favorite Count", 'Retweet Count', 'Tweet Link']
    top_content_tweets_df['Tweet Link'] = top_content_tweets_df['Tweet Link'].apply(get_url, 1)
    top_content_tweets_df['Tweet Link']= top_content_tweets_df['Tweet Link'].apply(lambda x: html.A(str("Link"),  href = str(x), target="_blank"))
    ER = round(df['Top_content'].sum()/df.shape[0], 1)
    link_percent = round(df['link'].sum()*100/df.shape[0], 1)
    rt_percent = round(df['RT_status'].sum()*100/df.shape[0], 1)
    pt_percent = round(df[df['RT_status']!=1][df['link']!=1][df['tweet_len']>=50].shape[0] * 100/ df.shape[0], 1)
    PR2 = df.shape[0]/max([1, (datetime.datetime.today() - datetime.datetime.strptime(str(df['created_at'].min()),'%a %b %d %H:%M:%S +0000 %Y')).days])

    usr = user_Details(status_full) #getting user info
    publication = int((usr['publication_daily'] + PR2)/2)

    top_tweet_url = [get_url(item) for item in top_content_id]

    all_tweets = " ".join([item['full_text'] for item in data if item['RT_status']!=1])
    
    
    #parsing all tweets to get top keywords, sectors, themes, etc
    tweet_keywords = keywordsf(all_tweets)
    tweet_sector1 = sector1f(all_tweets)
    tweet_sector2 = sector2f(all_tweets)
    tweet_themes = themef(all_tweets)
    tweet_company = companyf(all_tweets)
    tweet_hash = hash_pattern.findall(all_tweets)
    tweet_stocks = stock_pattern.findall(all_tweets)
    
    # getting unique counts
    total_uniq_hash = len(set(tweet_hash))
    total_uniq_stocks = len(set(tweet_stocks))
    total_uniq_keywords = len(set(tweet_keywords))
    total_uniq_IC = len(set(tweet_sector1))
    total_uniq_sectors = len(set(tweet_sector2))
    total_uniq_themes = len(set(tweet_themes))
    total_uniq_company = len(set(tweet_company))
    
    # getting top 10
    tweet_keywords = Counter(tweet_keywords).most_common(10)
    IC = Counter(tweet_sector1).most_common(10)
    tweet_sector2 = Counter(tweet_sector2).most_common(10)
    tweet_themes = Counter(tweet_themes).most_common(10)
    tweet_company = Counter(tweet_company).most_common(10)
    tweet_hash = Counter(tweet_hash).most_common(10)
    tweet_stocks = Counter(tweet_stocks).most_common(10)
    
    #putting in dataframe
    df_tweet_keywords = df_views(tweet_keywords, cols=['Top Keywords', 'Count'])
    df_IC = df_views(IC, cols = ['Top ICs', 'Count'])
    df_tweet_sector2 = df_views(tweet_sector2, cols= ['Top Sectors', 'Count'])
    df_tweet_themes = df_views(tweet_themes, cols = ['Top Themes', 'Count'])
    df_tweet_hash = df_views(tweet_hash, cols = ['Top HashTags', 'Count'])
    df_tweet_company = df_views(tweet_company, cols = ['Top Companies', 'Count'])
    df_tweet_stock = df_views(tweet_stocks, cols= ['Top Tickers', 'Count'])
    
    df_top_table = pd.concat([df_tweet_themes, df_tweet_sector2, df_IC, df_tweet_keywords,
                              df_tweet_hash, df_tweet_company, df_tweet_stock], axis = 1)
    
    total_tweets = df.shape[0]
    
    return (usr['name'], usr['location'], usr['verified'],
            usr['description'], 
            usr['followers'], usr['following'], usr['statuses_count'], usr['days_lived'],
            ER, publication, rt_percent, link_percent, pt_percent,
            top_tweet_url,
            tweet_keywords, IC, tweet_sector2, tweet_themes, tweet_hash, top_content_tweets_df,total_tweets, df_top_table, 
            total_uniq_themes, total_uniq_sectors, total_uniq_IC, total_uniq_keywords, 
            total_uniq_hash, total_uniq_company, total_uniq_stocks)



In [ ]:
# for testing purpose only
# %%time
# final_call(name = "Paytm")

In [ ]:
# building a dash app

In [ ]:
import dash
import dash_auth
import dash_table
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import base64
import re

def generate_table(dataframe, max_rows=100):
    return (
        # Header
        [html.Tr([html.Th(html.Small(col), style={"height":"10px", 'margin':'20px', 'padding':'2px', 'border': '1px solid grey'}) for col in dataframe.columns])] +

        # Body
        [html.Tr([
            html.Td(html.Small(dataframe.iloc[i, j]), style={"height":"10px", 'margin':'20px', 'padding':'1px', 'border': '1px solid grey'}) for j in range(dataframe.shape[1])
        ]) for i in range(min(len(dataframe), max_rows))]
    )


In [ ]:
VALID_USERNAME_PASSWORD_PAIRS = {
    'Iam': 'sidharth',
}


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

auth = dash_auth.BasicAuth(
    app,
    VALID_USERNAME_PASSWORD_PAIRS
)

app.layout = html.Div([
    html.H5(html.U('GlobalData: Twitter User Analysis'), style={'width': '95%', 'float': 'middle','display': 'inline-block', 'margin':'auto', 'text-align': 'center'}),
    html.Div([
        html.H6("Provide a handle:", style={'float': 'left','display': 'inline-block', 'margin':'auto'}),
        dcc.Input(id='cd', type='text', debounce=True, value="globaldataplc", style={'width': '40%', 'float': 'middle','display': 'inline-block', 'margin':'auto'})
        ],style={'width': '40%', 'float': 'middle','display': 'inline-block', 'margin':'auto'}),
    
    #html.H6(children='*************************************', style={'width': '100%', 'padding': '0 20'}), 
    
    html.Div([html.P(id='L1a')],style={'width': '95%', 'float': 'left', 'display': 'inline-block', 'margin':'auto', 
                                          'padding': '5px', 'border': '1px solid black'}),
    
    html.Div([html.P(id='L2a'),],style={'width': '95%', 'float': 'left', 'display': 'inline-block', 'margin':'auto', 
                                          'padding': '5px', 'border': '1px solid black'}),
    
    html.Div([html.P(id='L3a')],style={'width': '95%', 'float': 'left', 'display': 'inline-block', 'margin':'auto', 
                                          'padding': '5px', 'border': '1px solid black'}),
    
    html.Div([html.P(id='L4a')],
             style={'width': '95%', 'float': 'left', 'display': 'inline-block', 'margin':'auto', 
                                          'padding': '5px', 'border': '1px solid black'}),
    #added latter
    html.Div([html.P(id='L7a')],
             style={'width': '95%', 'float': 'left', 'display': 'inline-block', 'margin':'auto', 
                                          'padding': '5px', 'border': '1px solid black'}),
    
    
    html.H6(children='  ', style={'width': '100%', 'float': 'left', }),
    html.Div([html.Table(id='L5a')], style={'width': '95%', 'display': 'inline-block', 'margin':'auto', 
                                          'padding': '5px', 'border': '1px solid black'}),
    
    
    html.H6(children='Top Latest Tweets:-', style={'width': '100%', 'float': 'left', 'padding': '0 20'}), 
    html.Div([html.Table(id='L6a')], style={'width': '95%','display': 'inline-block', 'margin':'auto', 
                                          'padding': '5px', 'border': '1px solid black'})
    

    
    ])


@app.callback([Output('L1a', 'children'), 
               Output('L2a', 'children'), 
               Output('L3a', 'children'), 
               Output('L4a', 'children'), 
               Output('L5a', 'children'),
               Output('L6a', 'children'), 
               Output('L7a', 'children'),
              ],
              [Input('cd', 'value')])
def final_call1(cd):
    print("user search: ", cd)
    if len(cd.split())==1:
        usr_name, usr_location, usr_verified, usr_description, usr_followers, usr_following, usr_statuses_count, usr_days_lived, ER, publication, rt_percent, link_percent, pt_percent, top_tweet_url, tweet_keywords, IC, tweet_sector2, tweet_themes, tweet_hashr, top_content_tweets_df, total_tweets, df_top_table, total_uniq_themes, total_uniq_sectors, total_uniq_IC, total_uniq_keywords, total_uniq_hash, total_uniq_company, total_uniq_stocks = final_call(name = cd)

        L1 = ["Display Name: ", html.B(usr_name, style={"margin-right": "100px", "margin-left": "5px"}),
              "Location: ", html.B(usr_location, style={"margin-right": "100px", "margin-left": "5px"}),
              "Verified Account: ", html.B(str(usr_verified), style={"margin-right": "100px", "margin-left": "5px"}),
              "Total tweets Fetched: ", html.B(total_tweets, style={"margin-left": "5px"})]

        L2 = ["Description: ", html.I(usr_description, style={"margin-left": "5px"})]

        L3 = ["Followers: ", html.Big(html.B(usr_followers), style={"margin-right": "150px", "margin-left": "5px"}),
              "Following: ", html.Big(html.B(usr_following), style={"margin-right": "150px", "margin-left": "5px"}),
              "Total Status Posted: ", html.Big(html.B(usr_statuses_count), style={"margin-right": "150px", "margin-left": "5px"}),
              "Account age: ", html.Big(html.B(usr_days_lived), style={"margin-left": "5px", "margin-right": "2px"}), html.I("days", style={"margin-right": "150px"})]

        L4 = ["Retweet percentage: ",html.Big(html.B( rt_percent), style={"margin-right": "2px", "margin-left": "5px"}), html.I("%", style={"margin-right": "100px"}),
              "Link Sharing percentage: ", html.Big(html.B(link_percent), style={"margin-right": "2px", "margin-left": "5px"}), html.I("%", style={"margin-right": "100px"}),
              "Pure Thought percentage: ", html.Big(html.B(pt_percent), style={"margin-right": "2px", "margin-left": "5px"}), html.I("%", style={"margin-right": "100px"}),
              html.Br(),
              "Engagement Ratio: ", html.Big(html.B(ER), style={"margin-right": "5px", "margin-left": "5px"}), html.Small(html.I("((Retweet + Favourite) / Total Tweets))"), style={"margin-right": "100px", 'color': '#69737a'}), 
              "Publication Frequency: ", html.Big(html.B(publication), style={"margin-right": "5px", "margin-left": "5px"}), html.I("per day", style={"margin-right": "100px"}),
              "Total Influencer following[*Beta]: ", html.Big(html.B(infy_total), style={"margin-right": "5px", "margin-left": "5px"}),
              ]
        
        L7 = ["Total unique count of ~ ", 
              "Themes:",html.Big(html.B(total_uniq_themes), style={"margin-right": "50px", "margin-left": "5px"}),
              "Sectors:",html.Big(html.B(total_uniq_sectors), style={"margin-right": "50px", "margin-left": "5px"}),
              "ICs:",html.Big(html.B(total_uniq_IC), style={"margin-right": "50px", "margin-left": "5px"}),
              "Keywords:",html.Big(html.B(total_uniq_keywords), style={"margin-right": "50px", "margin-left": "5px"}),
              "HashTags:",html.Big(html.B(total_uniq_hash), style={"margin-right": "50px", "margin-left": "5px"}),
              "Companies:",html.Big(html.B(total_uniq_company), style={"margin-right": "50px", "margin-left": "5px"}),
              "Tickers:",html.Big(html.B(total_uniq_stocks), style={"margin-right": "50px", "margin-left": "5px"})]
        
        return [L1, L2, L3, L4, generate_table(df_top_table), generate_table(top_content_tweets_df), L7]
    else:
        L1 = ['Please Provide a Handle']
        L2 = []
        L3 = []
        L4 = []
        L7 = []
        L5 = []
        L8 = []
        return [L1, L2, L3, L4,
                L5,
                generate_table(pd.DataFrame()), L7, L8]


if __name__ == '__main__':
    app.debug = True
    app.run_server(port='', host='')
